In [ ]:
# -*- coding: utf-8 -*-
"""Speaker_diarization.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1A3ylQdfTohb3qqudczW39dP0oy6j4a1w
"""

# from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from pydub import AudioSegment
from yt_dlp import YoutubeDL
# from spectralcluster import SpectralClusterer
# from resemblyzer import sampling_rate
import os
import requests
import json
import time

def convert_webm_to_wav(input_file, output_file):
    # Load the WebM file using pydub
    audio = AudioSegment.from_file(input_file, format='webm')

    # Export the audio as WAV format
    audio.export(output_file, format='wav')

def split_audio_by_speaker(input_audio_file, segments):
    # Load the input audio using pydub
    audio = AudioSegment.from_file(input_audio_file)

    # Create an output directory
    output_directory = 'sample_data/data'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Create a dictionary to hold speaker segments
    speaker_segments = {}

    # Group segments by speaker
    for i, (speaker, start_time, end_time) in enumerate(segments):
        # If the speaker is not already in the dictionary, create a new list
        if speaker not in speaker_segments:
            speaker_segments[speaker] = []

        # Append the segment to the list of segments for the speaker
        speaker_segments[speaker].append((start_time, end_time))

    # Split the audio file into speaker segments
    for speaker, segments in speaker_segments.items():
        # Sort the segments by start time
        segments.sort(key=lambda x: x[0])

        # Generate the output filename
        output_file = os.path.join(output_directory, f"speaker_{speaker}.wav")

        # Concatenate the segments and export as a single audio file
        concatenated = AudioSegment.empty()
        for start_time, end_time in segments:
            segment = audio[start_time:end_time]
            concatenated = concatenated + segment

        concatenated.export(output_file, format="wav")


# Example usage
url = input("Enter the YouTube video URL: ")

audio_downloader = YoutubeDL({'format': 'bestaudio'})
video_info = audio_downloader.extract_info(url, download=False)
video_url = video_info['url']

# Download the video as WebM format
webm_file = audio_downloader.prepare_filename(video_info)
audio_downloader.process_info(video_info)
os.rename(audio_downloader.prepare_filename(video_info), webm_file)

# Convert the WebM file to WAV format
wav_file = 'sofia.wav'
convert_webm_to_wav(webm_file, wav_file)

# Upload the WAV file to AssemblyAI for transcription
base_url = "https://api.assemblyai.com/v2"
headers = {"authorization": "e2ce4a6e07c745668be2468dd9a34d30"}

with open(wav_file, "rb") as f:
    response = requests.post(base_url + "/upload", headers=headers, data=f)

upload_url = response.json()["upload_url"]
data = {"audio_url": upload_url, "speaker_labels": True}
url = base_url + "/transcript"
response = requests.post(url, json=data, headers=headers)
transcript_id = response.json()['id']
polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

while True:
    transcription_result = requests.get(polling_endpoint, headers=headers).json()
    transcription_segments = []

    if transcription_result['status'] == 'completed':
        if 'utterances' in transcription_result:
            utterances = transcription_result['utterances']
            for utterance in utterances:
                speaker = utterance['speaker']
                start = utterance['start']
                end = utterance['end']
                transcription_segments.append((speaker, start, end))

            # Print the transcription segments
            for segment in transcription_segments:
                print(segment)

            # Split the audio by speaker
            split_audio_by_speaker(wav_file, transcription_segments)
        else:
            print("No utterances found in the transcription.")
        break

    elif transcription_result['status'] == 'error':
        raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

Enter the YouTube video URL: https://www.youtube.com/watch?v=oaJGrv9FUh8
[youtube] Extracting URL: https://www.youtube.com/watch?v=oaJGrv9FUh8
[youtube] oaJGrv9FUh8: Downloading webpage
[youtube] oaJGrv9FUh8: Downloading ios player API JSON
[youtube] oaJGrv9FUh8: Downloading android player API JSON
[youtube] oaJGrv9FUh8: Downloading m3u8 information
[download] Destination: 8 Days In Real India [oaJGrv9FUh8].webm
[download] 100% of    8.60MiB in 00:00:00 at 37.67MiB/s  
('A', 330, 61120)
('B', 65270, 140098)
('A', 140274, 141098)
('B', 141184, 163170)
('A', 166390, 271026)
('C', 271128, 277458)
('A', 277554, 288886)
('C', 288998, 290090)
('A', 294110, 603420)


In [ ]:
!pip install pydub

In [ ]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 45.9 MB/s eta 0:00:00


- Mr beast  https://www.youtube.com/watch?v=-tObpcgCvDA
https://www.youtube.com/watch?v=1WEAJ-DFkHE
https://www.youtube.com/watch?v=6TEccm9Udkk
https://www.youtube.com/watch?v=Dc2ZRmuH5OM

Nas daily: https://www.youtube.com/watch?v=SJU-pVHkIIU
https://www.youtube.com/watch?v=w39hrxD7PWQ
https://www.youtube.com/watch?v=VxUoZi9LrDY
https://www.youtube.com/watch?v=w39hrxD7PWQ&t=18s
https://www.youtube.com/watch?v=lW0mHiQHx8c

sofia : https://www.youtube.com/watch?v=DZaoi-YKI5Q
https://www.youtube.com/watch?v=a-6UY-dHnNQ&list=RDQMVQYeS9fMIhI&start_radio=1
https://www.youtube.com/watch?v=C4khFFBJGSw
https://www.youtube.com/watch?v=jR-ubByAQkg

In [ ]:
import os
import json
import time
import requests
from pydub import AudioSegment
from pydub.utils import make_chunks
import speech_recognition as sr

def convert_webm_to_wav(input_file, output_file):
    # Load the WebM file using pydub
    audio = AudioSegment.from_file(input_file, format='webm')

    # Export the audio as WAV format
    audio.export(output_file, format='wav')

def split_audio_by_speaker(input_audio_file, segments):
    # Load the input audio using pydub
    audio = AudioSegment.from_file(input_audio_file)

    # Create an output directory
    output_directory = 'sample_data/data'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Create a dictionary to hold speaker segments
    speaker_segments = {}

    # Group segments by speaker
    for i, (speaker, start_time, end_time) in enumerate(segments):
        # If the speaker is not already in the dictionary, create a new list
        if speaker not in speaker_segments:
            speaker_segments[speaker] = []

        # Append the segment to the list of segments for the speaker
        speaker_segments[speaker].append((start_time, end_time))

    # Split the audio file into speaker segments
    for speaker, segments in speaker_segments.items():
        # Sort the segments by start time
        segments.sort(key=lambda x: x[0])

        # Generate the output filename
        output_file = os.path.join(output_directory, f"speaker_{speaker}.wav")

        # Concatenate the segments and export as a single audio file
        concatenated = AudioSegment.empty()
        for start_time, end_time in segments:
            segment = audio[start_time:end_time]
            concatenated = concatenated + segment

        concatenated.export(output_file, format="wav")


# Example usage
url = input("Enter the YouTube video URL: ")

audio_downloader = YoutubeDL({'format': 'bestaudio'})
video_info = audio_downloader.extract_info(url, download=False)
video_url = video_info['url']

# Download the video as WebM format
webm_file = audio_downloader.prepare_filename(video_info)
audio_downloader.process_info(video_info)
os.rename(audio_downloader.prepare_filename(video_info), webm_file)

# Convert the WebM file to WAV format
wav_file = 'emma.wav'
convert_webm_to_wav(webm_file, wav_file)

# Perform speech recognition using Google Web Speech API
recognizer = sr.Recognizer()
with sr.AudioFile(wav_file) as source:
    audio = recognizer.record(source)

# Transcribe the speech
try:
    text = recognizer.recognize_google(audio)
    print("Transcription:")
    print(text)
except sr.UnknownValueError:
    print("Google Web Speech API could not understand the audio.")
except sr.RequestError as e:
    print("Could not request results from Google Web Speech API; {0}".format(e))

# Optional: If you need to split audio by speaker based on transcriptions, you can add the logic here using the 'text' variable.


ModuleNotFoundError: ignored

In [ ]:
pip install speechrecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 28.0 MB/s eta 0:00:00


In [ ]:
import os
import requests
import speech_recognition as sr
from pydub import AudioSegment
from pyAudioAnalysis import audioSegmentation as aS

def convert_webm_to_wav(input_file, output_file):
    # Load the WebM file using pydub
    audio = AudioSegment.from_file(input_file, format='webm')

    # Export the audio as WAV format
    audio.export(output_file, format='wav')

# Function to transcribe audio using Google Web Speech Recognition API
def google_speech_recognition(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

    try:
        # Perform speech recognition using Google Web Speech API
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Google Web Speech API could not understand the audio.")
    except sr.RequestError as e:
        print("Could not request results from Google Web Speech API; {0}".format(e))

    return None

# Example usage
url = input("Enter the YouTube video URL: ")

audio_downloader = YoutubeDL({'format': 'bestaudio'})
video_info = audio_downloader.extract_info(url, download=False)
video_url = video_info['url']

# Download the video as WebM format
webm_file = audio_downloader.prepare_filename(video_info)
audio_downloader.process_info(video_info)
os.rename(audio_downloader.prepare_filename(video_info), webm_file)

# Convert the WebM file to WAV format
wav_file = 'sofia.wav'
convert_webm_to_wav(webm_file, wav_file)

# Perform speech recognition using Google Web Speech API
transcribed_text = google_speech_recognition(wav_file)

if transcribed_text:
    print("Transcription:")
    print(transcribed_text)

# Perform speaker diarization using pyAudioAnalysis
# You need to implement the speaker diarization using pyAudioAnalysis based on the transcribed text or audio segments.
# This step is not included in the provided example as it requires further processing beyond the scope of this response.


ModuleNotFoundError: ignored

In [ ]:
!pip install pyAudioAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264374 sha256=e52d17cd876f2cb7553ea5a82065b725a35865a046537d25cc7058d43cd0f7c3
  Stored in directory: /root/.cache/pip/wheels/a7/54/73/fa830689c2440d2c81ff175c60e374930ad1607a8881e0f43f
Successfully built pyAudioAnalysis


In [ ]:
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 2.5 MB/s eta 0:00:00
